# Energy consumption of single household with yearly seasonality

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_absolute_error

In [2]:
full_data = pd.read_csv("../../data/synthetic/ts_dayly_averaged.csv")
# full_data = pd.read_csv("../../data/synthetic/ts_weekly_averaged.csv")
print(len(full_data))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/synthetic/ts_dayly_averaged.csv'

In [ ]:
full_data.head()

In [ ]:
data_frame = full_data.drop(columns=['Index']).copy()

In [ ]:
plt.figure(figsize=(40,20), dpi=40)
plt.locator_params(axis='x', nbins=3)
x_ticks = np.arange(0, len(data_frame["Time"]), 200)
plt.xticks(x_ticks)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.plot(data_frame["Time"], data_frame["Energy"])
plt.xlabel("Date", fontsize=40)
plt.ylabel("Energy Consumption (kW-h)", fontsize=40)
plt.title("Energy Consumption Single French Household along 4 years", fontsize=40)
plt.show()

In [ ]:
# We observed from the above plot that the trend, seasonality and periodicity can be identified. The irregularity are mostly given by the outliers.

# Rolling Statistics


In [ ]:
rollmean = data_frame["Energy"].rolling(window=365).mean()
rollstd = data_frame["Energy"].rolling(window=365).std()
# rollmean = data_frame["Energy"].rolling(window=52).mean()
# rollstd = data_frame["Energy"].rolling(window=52).std()

In [ ]:
plt.figure(figsize=(40,20), dpi=40)
plt.locator_params(axis='x', nbins=3)
x_ticks = np.arange(0, len(full_data["Time"]), 200)
plt.xticks(x_ticks)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.plot(data_frame["Time"], data_frame["Energy"], label='Original', linewidth='5')
mean = plt.plot(data_frame["Time"], rollmean, '-p', color='red', label='Rolling Mean', linewidth='5')
std = plt.plot(data_frame["Time"], rollstd, '-p', color='white', label='Rolling Std', linewidth='5')
plt.xlabel("Date", fontsize=40)
plt.ylabel("Energy Consumption (kW-h)", fontsize=40)
plt.title("Energy Consumption Single French Household along 4 years", fontsize=40)
plt.legend(loc='best', fontsize=40)
plt.show()

In [ ]:
# Also we notice that the data show a stationary behaviour (there is not overall increase or decrease)
# The mean and standard deviation show that there is no significant trend across the yearly seasons

## Outliers treatment

In [ ]:
# Find better colors to visualize the boxplot
plt.figure(figsize=(30,20), dpi=40)
plt.boxplot(data_frame['Energy'])
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title("Outliers Identification", fontsize=40)
plt.show()
print(data_frame["Energy"].mean())

In [ ]:
energy_cutoff = 3.5
# The following condition returns a boolean array and the sum() call adds only the true conditions
outliers_number = (data_frame["Energy"] > energy_cutoff).sum()
outliers_density = outliers_number/len(data_frame["Energy"])
print(f"The number of outliers is {outliers_number} and the outliers density is: {outliers_density}")

In [ ]:
# Outliers removal
data_frame.loc[data_frame.Energy > energy_cutoff, "Energy"] = data_frame["Energy"].mean()
plt.figure(figsize=(30,20), dpi=40)
plt.boxplot(data_frame['Energy'])
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title("Outliers Removal", fontsize=40)
plt.show()

In [ ]:
plt.figure(figsize=(40,20), dpi=40)
plt.locator_params(axis='x', nbins=3)
x_ticks = np.arange(0, len(full_data["Time"]), 200)
plt.xticks(x_ticks)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.plot(data_frame["Time"], data_frame["Energy"], label='Original', linewidth='5')
mean = plt.plot(data_frame["Time"], rollmean, '-p', color='red', label='Rolling Mean', linewidth='5')
std = plt.plot(data_frame["Time"], rollstd, '-p', color='white', label='Rolling Std', linewidth='5')
plt.xlabel("Date", fontsize=40)
plt.ylabel("Energy Consumption (kW-h)", fontsize=40)
plt.title("Energy Consumption Single French Household along 4 years", fontsize=40)
plt.legend(loc='best', fontsize=40)
plt.show()

## Normalization, Split and Model Training

In [ ]:
# We generate new features from the timestap column
data_frame['Day'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).day
data_frame['DayOfWeek'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).dayofweek
data_frame['Quarter'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).quarter
data_frame['Month'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).month
data_frame['Year'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).year
data_frame['WeekOfYear'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).weekofyear
data_frame['DayOfYear'] = pd.DatetimeIndex(data_frame['Time'], dayfirst=True).dayofyear
y_pre = data_frame.Energy.values
# We will use Standard Scaling because the data is not evenly distributed
scaler = StandardScaler()
y = scaler.fit_transform(y_pre.reshape(-1, 1))
# The size of the training dataset
n_train = 1095   # 3 years (365 * 3 = 1095 days)
# n_train = 156   # 3 years (365 * 3 / 7 = 156 weeks)
data_frame.head()

In [ ]:
def eval_on_features(features, target, regressor):
    X_train, X_test = features[:n_train], features[n_train:]
    y_train, y_test = target[:n_train], target[n_train:]
    regressor.fit(X_train, y_train)
    print("Test-set R^2: {:.2f}".format(regressor.score(X_test, y_test)))
    y_pred = regressor.predict(X_test)
    y_pred_train = regressor.predict(X_train)
    plt.figure(figsize=(40,20))
    x_ticks = np.arange(0, len(full_data["Time"]), 200)
    plt.xticks(x_ticks)
    plt.xticks(fontsize=40)
    # plt.xticks(range(0, len(X_day_month), 200), fontsize=40, rotation=90, ha="left")
    plt.yticks(fontsize=40)
    plt.plot(range(n_train), y_train, label="train", linewidth=5)
    plt.plot(range(n_train, len(y_test) + n_train), y_test, '-', label="test", linewidth=5)
    plt.plot(range(n_train), y_pred_train, '--', label="prediction train", linewidth=5)
    plt.plot(range(n_train, len(y_test) + n_train), y_pred, '--', label="prediction test", linewidth=5)
    plt.legend(loc=(0.7,0.7), fontsize=40)
    print(mean_absolute_error(y_test, y_pred))

In [ ]:
# Model training
X_day_month = np.hstack([data_frame.Day.values.reshape(-1,1), 
                         data_frame.DayOfWeek.values.reshape(-1,1),
                         data_frame.Quarter.values.reshape(-1,1),
                         data_frame.Month.values.reshape(-1,1),
                         data_frame.Year.values.reshape(-1,1),
                         data_frame.WeekOfYear.values.reshape(-1,1),
                         data_frame.DayOfYear.values.reshape(-1,1)])
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
eval_on_features(X_day_month, y.ravel(), regressor)


In [ ]:
enc = OneHotEncoder()
X_day_month_onehot = enc.fit_transform(X_day_month).toarray()
eval_on_features(X_day_month_onehot, y.ravel(), Ridge())

## Conclusion
- Predictions graphically seem to be quite ok, but the R-scores still low?! why? (maybe wrong metric?)
- Include mean error to better describe the model